In [2]:
# import necessary packages 

import csv
import numpy as np 
import re
import time
import pickle
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
from collections import defaultdict
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import chromedriver_binary
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import StaleElementReferenceException

In [80]:
from lxml import etree 

#initialize empty dict to store links 
link_list = []

#define url to parse 
url = 'https://www.allbud.com/marijuana-strains/search'


from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-notifications')
driver = webdriver.Chrome (options = options)
driver.get(url)


#assign variable for pagination
page = 2  

#assign variable for looping        
i= 1
            
while page <235:
    #scroll to the bottom of the page and wait for the page to load up 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    soup = BeautifulSoup (driver.page_source, 'lxml')
    dom = etree.HTML(str(soup))
    
    #there were 40 products on each page. loops through the string to form an xpath for selenium to find. 
    for i in range(1, 41):
        xpath = '//*[@id="search-results"]/div[' + str(i) +']/article[1]/header/a'
        atag = dom.xpath(xpath)
        i+=1
        
        #get the href from the xpath and append to empty link list
        for link in atag:
            review_url = link.get('href')
            review_url = 'https://www.allbud.com' + review_url
            link_list.append(review_url) 
        
    #pagination- to go from page to page    
    driver.get('https://www.allbud.com/marijuana-strains/search?page=' + str(page))
    page+=1


driver.close()

len(link_list)

In [99]:
#to check if the list was created with all the links 
len(link_list)

9290

In [105]:
#loading list into a pickle for future reference 

import pickle

with open('link_list', 'wb') as all_links:
    pickle.dump(link_list, all_links)
    
with open('link_list', 'rb') as all_links:
    b = pickle.load(all_links)

b

['https://www.allbud.com/marijuana-strains/hybrid/sage-sativa-afghani-genetic-equilibrium',
 'https://www.allbud.com/marijuana-strains/hybrid/cannatonic',
 'https://www.allbud.com/marijuana-strains/sativa-dominant-hybrid/blue-dream',
 'https://www.allbud.com/marijuana-strains/sativa-dominant-hybrid/goji-og',
 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/green-love-potion',
 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/pineapple-skunk',
 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/master-kush',
 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/ace-of-spades',
 'https://www.allbud.com/marijuana-strains/sativa/durban-poison',
 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/blue-cheese',
 'https://www.allbud.com/marijuana-strains/indica-dominant-hybrid/gorilla-glue-4',
 'https://www.allbud.com/marijuana-strains/sativa-dominant-hybrid/jack-herer',
 'https://www.allbud.com/marijuana-strains/indica-

In [95]:
# code block to extract name, thc content, effects, may relieve, aromas, flavors, strain content, description
#and favorite counts from each individual product link. 

data = []

options = Options()
options.add_argument("--headless")
options.add_argument('--disable-notifications')

driver = webdriver.Chrome (options = options)


for link in link_list:
    driver.get(link)
    #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    #time.sleep(2)
    hun = BeautifulSoup(driver.page_source, 'lxml')
    dom = etree.HTML(str(soup))
    
    try:
        name = driver.find_element(By.XPATH, '//*[@id="strain_detail_accordion"]/div[@class="row"][1]//h1')
        name = name.text
    except:
        name = ''
    
    try:
        thc_content = driver.find_element(By.XPATH, '//*[@id="strain-info"]/div/div[2]/div[1]/h4[2]')
        thc_content = thc_content.text
    except:
        thc_content = ''
    
    try:
        effects = driver.find_element(By.XPATH, '//*[@id="positive-effects"]/div/div[2]')
        effects = effects.text.split(',')
    except:
        effects = []
    
    try:
        may_relieve = driver.find_element(By.XPATH, '//*[@id="relieved"]/div/div[2]')
        may_relieve = may_relieve.text.split(',')
    except:
        may_relieve = []
        
    try:
        flavors = driver.find_element(By.XPATH, '//*[@id="flavors"]/div/div[2]')
        flavors = flavors.text.split(',')
    except:
        flavors = []
    
    try:
        aromas = driver.find_element(By.XPATH, '//*[@id="aromas"]/div/div[2]')
        aromas = aromas.text.split(',')
    except:
        aromas = []
    
    try:
        strain_type = driver.find_element(By.XPATH, '//*[@id="strain-info"]/div/div[2]/div[1]/h4[1]/a')
        strain_type = strain_type.text
    except:
        strain_type = ''
    
    try:
        strain_comp = driver.find_element(By.XPATH, ' //*[@id="strain-info"]/div/div[2]/div[1]/h4[1]/span[2]')
        strain_comp = strain_comp.text
    
    except:
        strain_comp = ''
        
    try:
        rating = driver.find_element(By.XPATH, '//*[@id="strain_detail_accordion"]/div[1]/div[2]/span/span[1]')
        rating = rating.text
    except:
        rating = ''
    
    try:
        description = driver.find_element(By.XPATH, '//*[@id="strain-info"]/div/div[2]/span')
        description = description.text
    except:
        description = ''
    
    try:
        fav_count = driver.find_element(By.XPATH, "//*[@id='unfavoritestrain']")
        fav_count = fav_count.get_attribute('data-follow-count')
    except:
        fav_count = ''
        
    

    data.append({'name':name, 'strain_type': strain_type, 'strain_comp':strain_comp,
                'thc_content':thc_content, 'effect':effects, 'may_relieve':may_relieve,
                'aromas':aromas, 'flavors':flavors, 'rating':rating, 'description':description,
                'fav_count':fav_count})
    
    

    
#creating dataframe to store data. 
strains_df = pd.DataFrame(data)
strains_df.head()



,name,strain_type,strain_comp,thc_content,effect,may_relieve,aromas,flavors,rating,description,fav_count
0,S.A.G.E. (Sativa Afghani Genetic Equilibrium) ...,Hybrid,50% Sativa /50% Indica,THC: 19% - 28%,"[Energizing, Euphoria, Happy, Relaxing]","[ADD/ADHD, Anxiety, Chronic Pain, Depressio...","[Earthy, Herbal, Sage, Spicy, Woody]","[Sage, Spicy]",4.8,S.A.G.E is a popular marijuana strain that won...,621
1,Cannatonic Marijuana Strain,Hybrid,50% Sativa /50% Indica,"THC: 7% - 8%, CBD: 14%, CBN: 1%","[Calming, Happy, Mellow, Relaxing, Uplifting]","[Anxiety, Chronic Pain, Cramps, Headaches, ...","[Citrus, Earthy, Herbal, Lemon, Sour, Spi...","[Citrus, Herbal, Sour, Spicy, Sweet, Woody]",4.7,Cannatonic is an evenly balanced hybrid strain...,1535
2,Blue Dream Marijuana Strain,Sativa Dominant Hybrid,60% Sativa / 40% Indica,"THC: 17% - 24%, CBD: 2%, CBN: 1%","[Creative, Energizing, Euphoria, Happy]","[ADD/ADHD, Anxiety, Autism, Bipolar Disorde...","[Earthy, Fruity, Sweet, Vanilla]","[Berry, Blueberry, Fruity, Herbal, Sweet, ...",4.5,Blue Dream is a slightly sativa dominant hybri...,15731
3,Goji OG Marijuana Strain,Sativa Dominant Hybrid,80% Sativa / 20% Indica,"THC: 22% - 26%, CBD: 1%","[Creative, Energizing, Euphoria, Happy, Up...","[Chronic Pain, Depression, Gastrointestinal ...","[Cherry, Pungent, Sweet]","[Berry, Cherry, Strawberry, Sweet, Tropical]",4.5,"Named after the bright red Himalayan berry, th...",301
4,Green Love Potion Marijuana Strain,Indica Dominant Hybrid,80% Indica / 20% Sativa,THC: 11% - 12%,"[Aroused, Happy, Relaxing, Sleepy]","[Anxiety, Chronic Pain, Cramps, Depression,...","[Earthy, Floral, Lavender, Pungent, Sweet]","[Chemical, Herbal, Lavender, Menthol, Spic...",4.5,"Being an indica dominant hybrid strain, Green ...",60


In [96]:
#checking shape of data frame. 
strains_df.shape

(9290, 11)

In [97]:
#saving dataframe as csv to have copy of data. 
strains_df.to_csv('final all_bud_strains.csv')